<a href="https://colab.research.google.com/github/hardiashrestha/Team-19-aizenberGAARRS-/blob/main/U-net%20model%20for%20segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from keras.preprocessing.image import load_img, img_to_array
import os

image_paths = os.listdir('/content/drive/MyDrive/PH2 Dataset images')

mask_paths = os.listdir('/content/drive/MyDrive/PH2 Dataset images')
# print(mask_paths)
maskp = []
imgp = []
for path in mask_paths:
  maskp.append(f'/content/drive/MyDrive/PH2 Dataset images/{path}/{path}_lesion/{path}_lesion.bmp')
mask_paths = maskp
for path in image_paths:
  imgp.append(f'/content/drive/MyDrive/PH2 Dataset images/{path}/{path}_Dermoscopic_Image/{path}.bmp')
image_paths = imgp
# Image size
img_height, img_width = 192, 256

# Function to preprocess images
def preprocess_image(image):
    image = img_to_array(image)
    image = tf.image.resize(image, [img_height, img_width])
    image = image / 255.0  # Normalize pixel values
    return image

# Function to preprocess masks
def preprocess_mask(mask):
    mask = img_to_array(mask)
    mask = tf.image.resize(mask, [img_height, img_width])

    mask = tf.cast(mask > 0.5, tf.float32)  # Convert to binary mask
    mask = tf.image.rgb_to_grayscale(mask)
    return mask

# Load and preprocess images

images = [preprocess_image(load_img(path)) for path in image_paths]

# Load and preprocess masks

masks = [preprocess_mask(load_img(path)) for path in mask_paths]

# Create TensorFlow datasets
image_dataset = tf.data.Dataset.from_tensor_slices(images)
mask_dataset = tf.data.Dataset.from_tensor_slices(masks)

print(mask_dataset)
# Combine images and masks
dataset = tf.data.Dataset.zip((image_dataset, mask_dataset))


<_TensorSliceDataset element_spec=TensorSpec(shape=(192, 256, 1), dtype=tf.float32, name=None)>


In [ ]:
import tensorflow as tf
from keras.layers import *
import keras
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds


def unet_model(input_size=(192, 256, 3)):
    inputs = Input(input_size)

    # Encoder
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)

    # Decoder
    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4], axis=-1)
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(up6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv3], axis=-1)
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(up7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)

    up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2], axis=-1)
    conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(up8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv8)
    conv8 = BatchNormalization()(conv8)

    up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1], axis=-1)
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(up9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)
    conv9 = BatchNormalization()(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = tf.keras.models.Model(inputs=[inputs], outputs=[conv10])

    return model

model = unet_model()
model.summary()



# dataset, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)
# # Split the dataset into training and testing sets
# train_dataset = dataset['train']
# test_dataset = dataset['test']

# Define preprocessing function
# def preprocess_data(example):
#     image = tf.image.resize(example['image'], (256, 256)) / 255.0  # Resize and normalize images
#     mask = tf.image.resize(example['segmentation_mask'], (256, 256))  # Resize masks
#     mask = tf.cast(mask > 0, tf.float32)  # Take only the first channel (binary mask)
#     return image, mask

# from ph2 import dataset
# Apply preprocessing to the datasets
train_dataset = dataset
val_dataset = dataset


# Batch and shuffle the datasets
BATCH_SIZE = 32
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(buffer_size=1000)
val_dataset = val_dataset.batch(BATCH_SIZE)

# print(train_dataset)

# Define and compile the model
model = unet_model(input_size=(192, 256, 3))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset)

# # Evaluate the model
test_loss, test_accuracy = model.evaluate(val_dataset)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


# # Plot training history

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


model.saveweights('U-net Model')

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 192, 256, 3)]        0         []                            
                                                                                                  
 conv2d_38 (Conv2D)          (None, 192, 256, 64)         1792      ['input_3[0][0]']             
                                                                                                  
 batch_normalization_36 (Ba  (None, 192, 256, 64)         256       ['conv2d_38[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 conv2d_39 (Conv2D)          (None, 192, 256, 64)         36928     ['batch_normalization_36